<a href="https://colab.research.google.com/github/HollyIrvine/mgmt590_Team_DataNinja7_FinalProject/blob/main/Individual_Analyses/Operational_Analyst/PriyaChanduri_Operational_Analyst_DIVE_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prompt : I'm an Operational Excellence Analyst and I want to analyze an e-commerce dataset to identify areas for improvement. Provide an outline of the key features of the analysis using the DIVE framework (Discover, Investigate, Validate, Extend).

Key Features of the Analysis:

Operational Excellence Analyst:
•	Discover: Process efficiency, resource utilization, operational metrics
•	Investigate: What operational factors drive success? Where are bottlenecks?
•	Validate: Test operational improvement hypotheses
•	Extend: Operational optimization roadmap

Discover - Process Efficiency Metrics:

Shipping efficiency - average days from order to shipment
Regional performance - operational metrics by geographic region
Category analysis - volume and efficiency by product categories
Monthly trends - seasonal patterns and performance evolution


Prompt : Let's start with the "Discover" phase. I want to analyze shipping efficiency metrics from the EcomStoreSalesPipeline.EcomStoreDataTable BigQuery table. Calculate the overall average, median, min, and max shipping days, as well as the total orders and the percentage of orders shipped within 2 days (classified as 'Fast').

In [ ]:
#@markdown
%%bigquery results
WITH shipping_metrics AS (
  SELECT
    *,
    DATE_DIFF(ShipDate, OrderDate, DAY) AS shipping_days,
    CASE
      WHEN DATE_DIFF(ShipDate, OrderDate, DAY) <= 2 THEN 'Fast'
      WHEN DATE_DIFF(ShipDate, OrderDate, DAY) <= 4 THEN 'Standard'
      ELSE 'Slow'
    END AS shipping_performance_category
  FROM `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
  WHERE ShipDate IS NOT NULL
    AND OrderDate IS NOT NULL
    AND DATE_DIFF(ShipDate, OrderDate, DAY) >= 0
)

SELECT
  'Overall Shipping Performance' AS metric_category,
  ROUND(AVG(shipping_days), 1) AS avg_shipping_days,
  APPROX_QUANTILES(shipping_days, 100)[OFFSET(50)] AS median_shipping_days,
  MIN(shipping_days) AS min_shipping_days,
  MAX(shipping_days) AS max_shipping_days,
  COUNT(*) AS total_orders,
  COUNTIF(shipping_days <= 2) AS fast_orders,
  ROUND(COUNTIF(shipping_days <= 2) / COUNT(*) * 100, 1) AS fast_order_pct
FROM shipping_metrics;

prompt: Display the results of the previous BigQuery query.

In [ ]:
#@markdown
results

Prompt : Now, analyze shipping performance by ship mode. For each ShipMode, calculate the order count, average and median shipping days, average order value, average profit per order, profit margin percentage, number of fast deliveries (within 2 days), and the fast delivery rate percentage. Order the results by average shipping days ascending.

2. SHIPPING PERFORMANCE BY MODE

In [ ]:
#@markdown
%%bigquery results

WITH shipping_metrics AS (
  SELECT
    *,
    DATE_DIFF(ShipDate, OrderDate, DAY) AS shipping_days
  FROM `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
  WHERE ShipDate IS NOT NULL
    AND OrderDate IS NOT NULL
    AND DATE_DIFF(ShipDate, OrderDate, DAY) >= 0
)

SELECT
  ShipMode,
  COUNT(*) AS order_count,
  ROUND(AVG(shipping_days), 1) AS avg_shipping_days,
  APPROX_QUANTILES(shipping_days, 100)[OFFSET(50)] AS median_shipping_days,
  ROUND(AVG(Sales), 2) AS avg_order_value,
  ROUND(AVG(Profit), 2) AS avg_profit_per_order,
  ROUND(AVG(Profit) / AVG(Sales) * 100, 1) AS profit_margin_pct,
  COUNTIF(shipping_days <= 2) AS fast_deliveries,
  ROUND(COUNTIF(shipping_days <= 2) / COUNT(*) * 100, 1) AS fast_delivery_rate_pct
FROM shipping_metrics
GROUP BY ShipMode
ORDER BY avg_shipping_days ASC;

prompt: Display the results of the previous BigQuery query

In [ ]:
#@markdown
results

Prompt 6: Next, analyze regional operational efficiency. For each Region, calculate the total orders, unique customers, average orders per customer, average shipping days, total sales, total profit, profit margin percentage, average order value, number of loss-making orders (profit < 0), and the percentage of loss orders. Order the results by total sales descending.

3. REGIONAL OPERATIONAL EFFICIENCY

In [ ]:
#@markdown
%%bigquery results
WITH regional_ops AS (
  SELECT
    *,
    DATE_DIFF(ShipDate, OrderDate, DAY) AS shipping_days
  FROM `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
  WHERE ShipDate IS NOT NULL
    AND OrderDate IS NOT NULL
    AND DATE_DIFF(ShipDate, OrderDate, DAY) >= 0
)

SELECT
  Region,
  COUNT(*) AS total_orders,
  COUNT(DISTINCT CustomerID) AS unique_customers,
  ROUND(COUNT(*) / COUNT(DISTINCT CustomerID), 1) AS avg_orders_per_customer,
  ROUND(AVG(shipping_days), 1) AS avg_shipping_days,
  ROUND(SUM(Sales), 2) AS total_sales,
  ROUND(SUM(Profit), 2) AS total_profit,
  ROUND(SUM(Profit) / SUM(Sales) * 100, 1) AS profit_margin_pct,
  ROUND(AVG(Sales), 2) AS avg_order_value,
  COUNTIF(Profit < 0) AS loss_making_orders,
  ROUND(COUNTIF(Profit < 0) / COUNT(*) * 100, 1) AS loss_order_pct
FROM regional_ops
GROUP BY Region
ORDER BY total_sales DESC;

Prompt : Display the results of the previous BigQuery query.

In [ ]:
#@markdown
results

Prompt 8: Analyze product category operational metrics. For each Category, calculate the order volume, total units sold, average units per order, total revenue, total profit, profit margin percentage, average shipping days, average discount percentage, number of unprofitable orders (profit < 0), and the percentage of unprofitable orders. Order the results by total revenue descending.

4. PRODUCT CATEGORY OPERATIONAL METRICS

In [ ]:
#@markdown
%%bigquery results
WITH category_ops AS (
  SELECT
    *,
    DATE_DIFF(ShipDate, OrderDate, DAY) AS shipping_days
  FROM `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
  WHERE ShipDate IS NOT NULL
    AND OrderDate IS NOT NULL
)

SELECT
  Category,
  COUNT(*) AS order_volume,
  ROUND(SUM(Quantity), 0) AS total_units_sold,
  ROUND(AVG(Quantity), 1) AS avg_units_per_order,
  ROUND(SUM(Sales), 2) AS total_revenue,
  ROUND(SUM(Profit), 2) AS total_profit,
  ROUND(SUM(Profit) / SUM(Sales) * 100, 1) AS profit_margin_pct,
  ROUND(AVG(shipping_days), 1) AS avg_shipping_days,
  ROUND(AVG(Discount) * 100, 1) AS avg_discount_pct,
  COUNTIF(Profit < 0) AS unprofitable_orders,
  ROUND(COUNTIF(Profit < 0) / COUNT(*) * 100, 1) AS unprofitable_order_pct
FROM category_ops
GROUP BY Category
ORDER BY total_revenue DESC;

prompt : Display the results of the previous BigQuery query.

In [ ]:
#@markdown
results

QueryJob<project=, location=None, id=45a6cca7-c42f-45e1-ad55-a0f51e05f7e3>

Prompt 10: Analyze customer segment efficiency. For each Segment, calculate the total orders, unique customers, average orders per customer, total revenue, revenue per customer, average order value, profit margin percentage, average shipping days, and average discount rate percentage. Order the results by revenue per customer descending.

-- 5. CUSTOMER SEGMENT EFFICIENCY

In [ ]:
#@markdown
%%bigquery results
WITH segment_analysis AS (
  SELECT
    *,
    DATE_DIFF(ShipDate, OrderDate, DAY) AS shipping_days
  FROM `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
  WHERE ShipDate IS NOT NULL
    AND OrderDate IS NOT NULL
)

SELECT
  Segment,
  COUNT(*) AS total_orders,
  COUNT(DISTINCT CustomerID) AS unique_customers,
  ROUND(COUNT(*) / COUNT(DISTINCT CustomerID), 1) AS avg_orders_per_customer,
  ROUND(SUM(Sales), 2) AS total_revenue,
  ROUND(SUM(Sales) / COUNT(DISTINCT CustomerID), 2) AS revenue_per_customer,
  ROUND(AVG(Sales), 2) AS avg_order_value,
  ROUND(SUM(Profit) / SUM(Sales) * 100, 1) AS profit_margin_pct,
  ROUND(AVG(shipping_days), 1) AS avg_shipping_days,
  ROUND(AVG(Discount) * 100, 1) AS avg_discount_rate_pct
FROM segment_analysis
GROUP BY Segment
ORDER BY revenue_per_customer DESC;

Prompt 11: Display the results of the previous BigQuery query.

In [ ]:
#@markdown
results

QueryJob<project=, location=None, id=17c8ad70-e68a-4a21-8fa8-c95d4ed6609e>

Prompt : Analyze monthly operational trends. For each order_year and order_month, calculate the order count, monthly revenue, monthly profit, profit margin percentage, average shipping days, number of fast deliveries (within 2 days), and the fast delivery rate percentage. Order the results by year and month.

In [ ]:
#@markdown
-- 6. MONTHLY OPERATIONAL TRENDS

In [ ]:
#@markdown
%%bigquery results
WITH monthly_trends AS (
  SELECT
    *,
    EXTRACT(YEAR FROM OrderDate) AS order_year,
    EXTRACT(MONTH FROM OrderDate) AS order_month,
    DATE_DIFF(ShipDate, OrderDate, DAY) AS shipping_days
  FROM `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
  WHERE OrderDate IS NOT NULL
)

SELECT
  order_year,
  order_month,
  FORMAT_DATE('%B', DATE(order_year, order_month, 1)) AS month_name,
  COUNT(*) AS order_count,
  ROUND(SUM(Sales), 2) AS monthly_revenue,
  ROUND(SUM(Profit), 2) AS monthly_profit,
  ROUND(SUM(Profit) / SUM(Sales) * 100, 1) AS profit_margin_pct,
  ROUND(AVG(shipping_days), 1) AS avg_shipping_days,
  COUNTIF(shipping_days <= 2) AS fast_deliveries,
  ROUND(COUNTIF(shipping_days <= 2) / COUNT(*) * 100, 1) AS fast_delivery_rate_pct
FROM monthly_trends
GROUP BY order_year, order_month
ORDER BY order_year, order_month;

In [ ]:
#@markdown
prompt 13: Display the results of the previous BigQuery query.

In [ ]:
#@markdown
results

Prompt 14: Identify operational bottlenecks by looking at slow-shipping product sub-categories. For each Category and Sub-Category, calculate the order count, average shipping days, total sales, and profit margin percentage. Filter for sub-categories with at least 50 orders. Categorize the bottleneck priority based on average shipping days (High > 5 days, Medium > 3 days, Low <= 3 days). Order by average shipping days descending, then total sales descending.

-- 7. BOTTLENECK IDENTIFICATION

In [ ]:
#@markdown
%%bigquery results
-- Identify slow-shipping product categories
WITH shipping_bottlenecks AS (
  SELECT
    Category,
    `Sub-Category`,
    COUNT(*) AS order_count,
    ROUND(AVG(DATE_DIFF(ShipDate, OrderDate, DAY)), 1) AS avg_shipping_days,
    ROUND(SUM(Sales), 2) AS total_sales,
    ROUND(SUM(Profit) / SUM(Sales) * 100, 1) AS profit_margin_pct
  FROM `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
  WHERE ShipDate IS NOT NULL
    AND OrderDate IS NOT NULL
  GROUP BY Category, `Sub-Category`
  HAVING COUNT(*) >= 50  -- Filter for significant volume
)
SELECT
  Category,
  `Sub-Category`,
  order_count,
  avg_shipping_days,
  total_sales,
  profit_margin_pct,
  CASE
    WHEN avg_shipping_days > 5 THEN 'High Priority - Slow Shipping'
    WHEN avg_shipping_days > 3 THEN 'Medium Priority - Standard Shipping'
    ELSE 'Low Priority - Fast Shipping'
  END AS bottleneck_priority
FROM shipping_bottlenecks
ORDER BY avg_shipping_days DESC, total_sales DESC;

In [ ]:
#@markdown
results

Prompt 16: Analyze profit efficiency by identifying high-volume, low-profit scenarios at the sub-category level. For each Category and Sub-Category, calculate the order count, total sales, total profit, profit margin percentage, average discount percentage, number of loss orders (profit < 0), and the percentage of loss orders. Filter for sub-categories with at least 20 orders. Categorize the optimization priority based on profit margin percentage (Critical < 0, High < 10, Medium < 20, Low >= 20). Include the average order value. Order by profit margin percentage ascending, then total sales descending.

-- 8. PROFIT EFFICIENCY ANALYSIS

In [ ]:
#@markdown
%%bigquery results
-- Identify high-volume, low-profit scenarios
WITH profit_efficiency AS (
  SELECT
    Category,
    `Sub-Category`,
    COUNT(*) AS order_count,
    ROUND(SUM(Sales), 2) AS total_sales,
    ROUND(SUM(Profit), 2) AS total_profit,
    ROUND(SUM(Profit) / SUM(Sales) * 100, 1) AS profit_margin_pct,
    ROUND(AVG(Discount) * 100, 1) AS avg_discount_pct,
    COUNTIF(Profit < 0) AS loss_orders,
    ROUND(COUNTIF(Profit < 0) / COUNT(*) * 100, 1) AS loss_order_pct
  FROM `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
  GROUP BY Category, `Sub-Category`
  HAVING COUNT(*) >= 20
)

SELECT
  *,
  CASE
    WHEN profit_margin_pct < 0 THEN 'Critical - Losing Money'
    WHEN profit_margin_pct < 10 THEN 'High Priority - Low Margin'
    WHEN profit_margin_pct < 20 THEN 'Medium Priority - Moderate Margin'
    ELSE 'Low Priority - Good Margin'
  END AS optimization_priority,
  ROUND(total_sales / order_count, 2) AS avg_order_value
FROM profit_efficiency
ORDER BY profit_margin_pct ASC, total_sales DESC;

In [ ]:
#@markdown
results

 Prompt: Create a comprehensive operational dashboard summary query. This single query should provide an overview of key business metrics: total orders, unique customers, unique order transactions, total revenue, total profit, overall profit margin percentage, average order value, average shipping days, fast deliveries, fast delivery rate percentage, unprofitable orders, and unprofitable order percentage

-- 9. COMPREHENSIVE OPERATIONAL DASHBOARD QUERY

In [ ]:
#@markdown
%%bigquery results
-- Single query for executive dashboard
WITH operational_summary AS (
  SELECT
    *,
    DATE_DIFF(ShipDate, OrderDate, DAY) AS shipping_days,
    EXTRACT(YEAR FROM OrderDate) AS order_year,
    EXTRACT(MONTH FROM OrderDate) AS order_month
  FROM `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
  WHERE OrderDate IS NOT NULL AND ShipDate IS NOT NULL
)

SELECT
  'Business Overview' AS section,
  COUNT(*) AS total_orders,
  COUNT(DISTINCT CustomerID) AS unique_customers,
  COUNT(DISTINCT OrderID) AS unique_order_transactions,
  ROUND(SUM(Sales), 2) AS total_revenue,
  ROUND(SUM(Profit), 2) AS total_profit,
  ROUND(SUM(Profit) / SUM(Sales) * 100, 1) AS overall_profit_margin_pct,
  ROUND(AVG(Sales), 2) AS avg_order_value,
  ROUND(AVG(shipping_days), 1) AS avg_shipping_days,
  COUNTIF(shipping_days <= 2) AS fast_deliveries,
  ROUND(COUNTIF(shipping_days <= 2) / COUNT(*) * 100, 1) AS fast_delivery_rate_pct,
  COUNTIF(Profit < 0) AS unprofitable_orders,
  ROUND(COUNTIF(Profit < 0) / COUNT(*) * 100, 1) AS unprofitable_order_pct
FROM operational_summary;

In [ ]:
#@markdown
results

Prompt : Create a BigQuery view named Operational_Analyst_demand_forecasting_model for demand forecasting with time series features. Include features like daily orders, daily units, daily sales, average processing days, time features (year, month, day of week, quarter, day of year, holiday season, weekend, week of month), moving averages (7-day and 30-day orders), lag features (1, 7, and 30 days for orders), volatility (30-day orders), trend direction, seasonal index, growth rate, a simple trend-based forecast (7 days ahead), and confidence intervals based on volatility. Also, include a capacity recommendation based on the forecast and a forecast accuracy error rate. Filter for orders with valid ship dates and order dates, and where 7-day moving average is not null. Order by region, category, and order date descending.


-- MODEL 1: DEMAND FORECASTING WITH TIME SERIES

In [ ]:
#@markdown
%%bigquery results

-- Create comprehensive time series for demand prediction
CREATE OR REPLACE table  `mgmt599-final-project.EcomStoreSalesPipeline.Operational_Analyst_demand_forecasting_model` AS
WITH time_series_base AS (
  SELECT
    OrderDate AS order_date,
    Region,
    Category,
    ShipMode,
    COUNT(*) AS daily_orders,
    SUM(Quantity) AS daily_units,
    SUM(Sales) AS daily_sales,
    AVG(DATE_DIFF(ShipDate,
                  OrderDate, DAY)) AS avg_processing_days
  FROM `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
  WHERE OrderDate IS NOT NULL AND ShipDate IS NOT NULL
  GROUP BY order_date, Region, Category, ShipMode
),

time_features AS (
  SELECT
    *,
    EXTRACT(YEAR FROM order_date) AS order_year,
    EXTRACT(MONTH FROM order_date) AS order_month,
    EXTRACT(DAYOFWEEK FROM order_date) AS day_of_week,
    EXTRACT(QUARTER FROM order_date) AS order_quarter,
    EXTRACT(DAYOFYEAR FROM order_date) AS day_of_year,

    -- Seasonality indicators
    CASE
      WHEN EXTRACT(MONTH FROM order_date) IN (11, 12, 1) THEN 1
      ELSE 0
    END AS is_holiday_season,

    CASE
      WHEN EXTRACT(DAYOFWEEK FROM order_date) IN (1, 7) THEN 1
      ELSE 0
    END AS is_weekend,

    -- Business cycle indicators
    CASE
      WHEN EXTRACT(DAY FROM order_date) BETWEEN 1 AND 7 THEN 'Week1'
      WHEN EXTRACT(DAY FROM order_date) BETWEEN 8 AND 14 THEN 'Week2'
      WHEN EXTRACT(DAY FROM order_date) BETWEEN 15 AND 21 THEN 'Week3'
      ELSE 'Week4'
    END AS week_of_month
  FROM time_features
),

moving_averages AS (
  SELECT
    *,
    -- Moving averages for trend analysis
    AVG(daily_orders) OVER (
      PARTITION BY Region, Category
      ORDER BY order_date
      ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS orders_7day_ma,

    AVG(daily_orders) OVER (
      PARTITION BY Region, Category
      ORDER BY order_date
      ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
    ) AS orders_30day_ma,

    -- Lag features for autoregression
    LAG(daily_orders, 1) OVER (
      PARTITION BY Region, Category
      ORDER BY order_date
    ) AS orders_lag_1,

    LAG(daily_orders, 7) OVER (
      PARTITION BY Region, Category
      ORDER BY order_date
    ) AS orders_lag_7,

    LAG(daily_orders, 30) OVER (
      PARTITION BY Region, Category
      ORDER BY order_date
    ) AS orders_lag_30,

    -- Volatility measures
    STDDEV(daily_orders) OVER (
      PARTITION BY Region, Category
      ORDER BY order_date
      ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
    ) AS orders_30day_volatility
  FROM time_features
),

predictive_features AS (
  SELECT
    *,
    -- Trend indicators
    CASE
      WHEN orders_7day_ma > orders_30day_ma * 1.1 THEN 'Upward'
      WHEN orders_7day_ma < orders_30day_ma * 0.9 THEN 'Downward'
      ELSE 'Stable'
    END AS trend_direction,

    -- Seasonal adjustment
    daily_orders / NULLIF(orders_30day_ma, 0) AS seasonal_index,

    -- Growth rate
    CASE
      WHEN orders_lag_30 > 0 THEN
        (daily_orders - orders_lag_30) / orders_lag_30 * 100
      ELSE 0
    END AS growth_rate_30day,

    -- Forecast using linear regression components
    -- Simple trend-based forecast (next 7 days)
    orders_7day_ma +
    (orders_7day_ma - LAG(orders_7day_ma, 7) OVER (
      PARTITION BY Region, Category ORDER BY order_date
    )) AS trend_forecast_7day
  FROM moving_averages
)

SELECT
  order_date,
  Region,
  Category,
  ShipMode,
  daily_orders AS actual_orders,
  ROUND(orders_7day_ma, 1) AS ma_7day,
  ROUND(orders_30day_ma, 1) AS ma_30day,
  ROUND(orders_30day_volatility, 2) AS volatility_30day,
  trend_direction,
  ROUND(seasonal_index, 2) AS seasonal_factor,
  ROUND(growth_rate_30day, 1) AS growth_rate_pct,
  ROUND(trend_forecast_7day, 1) AS forecast_7day_ahead,

  -- Confidence intervals (using volatility)
  ROUND(trend_forecast_7day - (orders_30day_volatility * 1.96), 1) AS forecast_lower_ci,
  ROUND(trend_forecast_7day + (orders_30day_volatility * 1.96), 1) AS forecast_upper_ci,

  -- Resource planning indicators
  CASE
    WHEN trend_forecast_7day > orders_30day_ma * 1.3 THEN 'Scale Up - 30%+ Increase Expected'
    WHEN trend_forecast_7day > orders_30day_ma * 1.15 THEN 'Scale Up - 15%+ Increase Expected'
    WHEN trend_forecast_7day < orders_30day_ma * 0.85 THEN 'Scale Down - 15%+ Decrease Expected'
    WHEN trend_forecast_7day < orders_30day_ma * 0.7 THEN 'Scale Down - 30%+ Decrease Expected'
    ELSE 'Maintain Current Capacity'
  END AS capacity_recommendation,

  -- Forecast accuracy score (how well recent forecasts performed)
  ABS(daily_orders - LAG(trend_forecast_7day, 7) OVER (
    PARTITION BY Region, Category ORDER BY order_date
  )) / NULLIF(daily_orders, 0) AS forecast_error_rate

FROM predictive_features
WHERE orders_7day_ma IS NOT NULL
ORDER BY Region, Category, order_date DESC;

In [ ]:
#@markdown
results

QueryJob<project=, location=None, id=4d3e360d-cd86-40fe-a103-aaf5dd70a775>

Prompt: Create a BigQuery view named Operationa_analyst_processing_time_prediction_model for predicting processing times. Include the actual processing days, order complexity features (size, value, discount category), time features (order month, day of week, holiday season), and baseline processing time metrics (average, median, percentiles 75 and 90, std deviation, sample size) grouped by region, category, sub-category, ship mode, order size, and order value category (filter groups with sample size >= 10). Calculate an adjusted prediction based on holiday season, weekend, and high discount. Include confidence intervals, prediction error and error rate (for historical data), processing risk category, and resource allocation recommendation. Filter for valid ship dates and order dates, and processing days between 0 and 30. Order by adjusted prediction descending, then order date descending.

In [ ]:
#@markdown
%%bigquery
-- ===============================================
-- MODEL 2: PROCESSING TIME PREDICTION MODEL
-- ===============================================

-- Predict processing times for capacity planning
CREATE OR REPLACE table `mgmt599-final-project.EcomStoreSalesPipeline.Operationa_analyst_processing_time_prediction_model` AS
WITH processing_features AS (
  SELECT
    *,
    DATE_DIFF(ShipDate, OrderDate, DAY) AS actual_processing_days,

    -- Order complexity features
    CASE
      WHEN Quantity = 1 THEN 'Single'
      WHEN Quantity <= 3 THEN 'Small'
      WHEN Quantity <= 5 THEN 'Medium'
      ELSE 'Large'
    END AS order_size_category,

    CASE
      WHEN Sales <= 100 THEN 'Low'
      WHEN Sales <= 500 THEN 'Medium'
      WHEN Sales <= 1000 THEN 'High'
      ELSE 'Premium'
    END AS order_value_category,

    CASE
      WHEN Discount = 0 THEN 'No_Discount'
      WHEN Discount <= 0.1 THEN 'Low_Discount'
      WHEN Discount <= 0.3 THEN 'Medium_Discount'
      ELSE 'High_Discount'
    END AS discount_category,

    -- Time features
    EXTRACT(MONTH FROM OrderDate) AS order_month,
    EXTRACT(DAYOFWEEK FROM OrderDate) AS order_day_of_week,

    CASE
      WHEN EXTRACT(MONTH FROM OrderDate) IN (11, 12, 1) THEN 1
      ELSE 0
    END AS is_holiday_season

  FROM `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
  WHERE ShipDate IS NOT NULL
    AND OrderDate IS NOT NULL
    AND DATE_DIFF(ShipDate, OrderDate, DAY) >= 0
    AND DATE_DIFF(ShipDate, OrderDate, DAY) <= 30
),

category_baselines AS (
  SELECT
    Region,
    Category,
    `Sub-Category`,
    ShipMode,
    order_size_category,
    order_value_category,
    AVG(actual_processing_days) AS avg_processing_days,
    STDDEV(actual_processing_days) AS std_processing_days,
    COUNT(*) AS sample_size,
    APPROX_QUANTILES(actual_processing_days, 100)[OFFSET(50)] AS median_processing_days,
    APPROX_QUANTILES(actual_processing_days, 100)[OFFSET(75)] AS p75_processing_days,
    APPROX_QUANTILES(actual_processing_days, 100)[OFFSET(90)] AS p90_processing_days
  FROM processing_features
  GROUP BY Region, Category, `Sub-Category`, ShipMode, order_size_category, order_value_category
  HAVING COUNT(*) >= 10  -- Ensure statistical significance
),

predictions AS (
  SELECT
    pf.*,
    cb.avg_processing_days AS baseline_prediction,
    cb.std_processing_days AS prediction_uncertainty,
    cb.median_processing_days,
    cb.p75_processing_days,
    cb.p90_processing_days,
    cb.sample_size,

    -- Adjust baseline prediction based on additional factors
    cb.avg_processing_days *
    CASE
      -- Holiday season adjustment
      WHEN pf.is_holiday_season = 1 THEN 1.2
      -- Weekend adjustment
      WHEN pf.order_day_of_week IN (1, 7) THEN 1.1
      -- High discount complexity adjustment
      WHEN pf.discount_category = 'High_Discount' THEN 1.15
      ELSE 1.0
    END AS adjusted_prediction,

    -- Confidence intervals
    cb.avg_processing_days - (cb.std_processing_days * 1.96) AS prediction_lower_ci,
    cb.avg_processing_days + (cb.std_processing_days * 1.96) AS prediction_upper_ci

  FROM processing_features pf
  LEFT JOIN category_baselines cb
    ON pf.Region = cb.Region
    AND pf.Category = cb.Category
    AND pf.`Sub-Category` = cb.`Sub-Category`
    AND pf.ShipMode = cb.ShipMode
    AND pf.order_size_category = cb.order_size_category
    AND pf.order_value_category = cb.order_value_category
)

SELECT
  RowID,
  OrderID,
  Region,
  Category,
  `Sub-Category`,
  ShipMode,
  order_size_category,
  order_value_category,
  discount_category,
  actual_processing_days,
  ROUND(baseline_prediction, 1) AS predicted_processing_days,
  ROUND(adjusted_prediction, 1) AS adjusted_predicted_days,
  ROUND(prediction_uncertainty, 2) AS prediction_std_dev,
  ROUND(prediction_lower_ci, 1) AS prediction_lower_bound,
  ROUND(prediction_upper_ci, 1) AS prediction_upper_bound,

  -- Prediction accuracy (for historical data)
  ABS(actual_processing_days - baseline_prediction) AS prediction_error,
  ABS(actual_processing_days - baseline_prediction) / NULLIF(actual_processing_days, 0) AS prediction_error_rate,

  -- Risk categories
  CASE
    WHEN adjusted_prediction > 5 THEN 'High Risk - Slow Processing Expected'
    WHEN adjusted_prediction > 3 THEN 'Medium Risk - Standard Processing'
    ELSE 'Low Risk - Fast Processing Expected'
  END AS processing_risk_category,

  -- Resource allocation recommendations
  CASE
    WHEN adjusted_prediction > 4 AND order_value_category IN ('High', 'Premium') THEN 'Priority Queue - Expedite'
    WHEN adjusted_prediction > 5 THEN 'Additional Resources Needed'
    WHEN adjusted_prediction > 3 THEN 'Standard Resource Allocation'
    ELSE 'Standard Processing Track'
  END AS resource_recommendation,

  sample_size

FROM predictions
WHERE baseline_prediction IS NOT NULL
ORDER BY adjusted_prediction DESC, OrderDate DESC;

In [ ]:
#@markdown

## DIVE Analysis of the E-commerce Operations Notebook

This analysis provides a comprehensive overview of the notebook's findings based on the **Discover, Investigate, Validate, Extend (DIVE)** framework.

### 1. Discover: Key Operational Insights

The initial queries have uncovered several critical patterns and performance metrics:

*   **Overall Shipping Performance**: The average shipping time is **4.0 days**, with only **22.2%** of orders qualifying as 'Fast' (shipped within 2 days). This suggests room for improvement in overall logistics speed.
*   **Shipping Mode Disparities**: 'Standard Class' is the most used mode but also the slowest, with an average of **5.0 days** and **0%** fast deliveries. In contrast, 'Same Day' and 'First Class' are significantly faster.
*   **Regional Performance Gaps**: The **West** region leads in sales and has a strong profit margin (15%). The **Central** region is a major concern, with the lowest profit margin (**8%**) and the highest rate of loss-making orders (**31.9%**).
*   **Product Category Profitability Issues**: The **Furniture** category, despite generating significant revenue, has a critically low profit margin of just **2.5%**. This is largely driven by a high percentage of unprofitable orders (33.6%) and high average discounts (17.4%). Technology and Office Supplies are much more profitable (17.4% and 17.1% margins, respectively).
*   **Bottlenecks Identified**: The analysis pinpoints specific sub-categories like 'Tables' and 'Bookcases' as consistent loss-makers, while others like 'Supplies' also have negative profit margins.

### 2. Investigate: Probing for Root Causes

The discoveries raise important questions that require deeper investigation:

*   **Central Region's Low Profitability**: *Why* is the Central region underperforming so drastically? We should investigate if this is due to higher shipping costs, a different product mix, more aggressive discounting strategies, or higher rates of product returns in this region.
*   **Furniture's Profitability Drain**: *What* is driving the high rate of unprofitable orders in the Furniture category? Is it due to the cost of goods, high damage rates during shipping, or the heavy discounts being offered? The query on profit efficiency strongly suggests discounts are a major factor.
*   **'Standard Class' Inefficiency**: *Why* is the most common shipping method the least efficient? Is this a trade-off for lower costs? An analysis comparing the shipping cost vs. profit margin for each mode would be insightful.

### 3. Validate: Confirming the Hypotheses

Before taking action, we should validate these initial findings:

*   **Drill Down into Problem Areas**: We can validate the regional and category issues by performing more granular queries. For the Central region, we can analyze performance by **State** or **City**. For the Furniture category, we can analyze the profitability of every single **product** within the 'Tables' and 'Bookcases' sub-categories.
*   **Statistical Significance**: The queries correctly filter for a minimum number of orders (e.g., `HAVING COUNT(*) >= 20`) to ensure the insights are based on a reliable sample size. This is a good practice that validates the significance of the findings.
*   **Model Backtesting**: The demand forecasting model created in the final step can be validated by **backtesting** it against historical data to check its prediction accuracy before using it for future planning.

### 4. Extend: Future Analysis and Strategic Actions

The notebook provides a strong foundation for several high-impact extensions:

*   **Build and Train ML Models**: The two views created (`Operational_Analyst_demand_forecasting_model` and `processing_time_prediction_model`) are feature-rich tables ready for machine learning. The next logical step is to use **BigQuery ML** to train a `ARIMA_PLUS` model for demand forecasting and a `BOOSTED_TREE_REGRESSOR` model for predicting processing times. This would shift the analysis from descriptive to predictive.
*   **Develop an Interactive Dashboard**: Connect this BigQuery project to **Looker Studio** or another BI tool. This would create a dynamic dashboard where stakeholders can filter by region, category, or date to explore the data and track KPIs in real-time.
*   **Actionable Business Recommendations**: Based on the validated insights, we can propose concrete actions:
    *   **Pricing Review**: Conduct a review of the pricing and discount strategy for the **Furniture** category, especially for loss-making sub-categories.
    *   **Regional Operational Audit**: Initiate an audit of the **Central** region's operations to identify the root causes of its low profitability.
    *   **Logistics Optimization**: Re-evaluate the carrier contracts and logistics for the **'Standard Class'** shipping mode to improve its speed and efficiency.

Prompt: Summarize the findings of the notebook using the DIVE framework (Discover, Investigate, Validate, Extend) and save the analysis as a markdown file.

In [ ]:
from google.colab import files

# The content of the DIVE analysis markdown cell
dive_content = """## DIVE Analysis of the E-commerce Operations Notebook

This analysis provides a comprehensive overview of the notebook's findings based on the **Discover, Investigate, Validate, Extend (DIVE)** framework.

### 1. Discover: Key Operational Insights

The initial queries have uncovered several critical patterns and performance metrics:

*   **Overall Shipping Performance**: The average shipping time is **4.0 days**, with only **22.2%** of orders qualifying as 'Fast' (shipped within 2 days). This suggests room for improvement in overall logistics speed.
*   **Shipping Mode Disparities**: 'Standard Class' is the most used mode but also the slowest, with an average of **5.0 days** and **0%** fast deliveries. In contrast, 'Same Day' and 'First Class' are significantly faster.
*   **Regional Performance Gaps**: The **West** region leads in sales and has a strong profit margin (15%). The **Central** region is a major concern, with the lowest profit margin (**8%**) and the highest rate of loss-making orders (**31.9%**).
*   **Product Category Profitability Issues**: The **Furniture** category, despite generating significant revenue, has a critically low profit margin of just **2.5%**. This is largely driven by a high percentage of unprofitable orders (33.6%) and high average discounts (17.4%). Technology and Office Supplies are much more profitable (17.4% and 17.1% margins, respectively).
*   **Bottlenecks Identified**: The analysis pinpoints specific sub-categories like 'Tables' and 'Bookcases' as consistent loss-makers, while others like 'Supplies' also have negative profit margins.

### 2. Investigate: Probing for Root Causes

The discoveries raise important questions that require deeper investigation:

*   **Central Region's Low Profitability**: *Why* is the Central region underperforming so drastically? We should investigate if this is due to higher shipping costs, a different product mix, more aggressive discounting strategies, or higher rates of product returns in this region.
*   **Furniture's Profitability Drain**: *What* is driving the high rate of unprofitable orders in the Furniture category? Is it due to the cost of goods, high damage rates during shipping, or the heavy discounts being offered? The query on profit efficiency strongly suggests discounts are a major factor.
*   **'Standard Class' Inefficiency**: *Why* is the most common shipping method the least efficient? Is this a trade-off for lower costs? An analysis comparing the shipping cost vs. profit margin for each mode would be insightful.

### 3. Validate: Confirming the Hypotheses

Before taking action, we should validate these initial findings:

*   **Drill Down into Problem Areas**: We can validate the regional and category issues by performing more granular queries. For the Central region, we can analyze performance by **State** or **City**. For the Furniture category, we can analyze the profitability of every single **product** within the 'Tables' and 'Bookcases' sub-categories.
*   **Statistical Significance**: The queries correctly filter for a minimum number of orders (e.g., `HAVING COUNT(*) >= 20`) to ensure the insights are based on a reliable sample size. This is a good practice that validates the significance of the findings.
*   **Model Backtesting**: The demand forecasting model created in the final step can be validated by **backtesting** it against historical data to check its prediction accuracy before using it for future planning.

### 4. Extend: Future Analysis and Strategic Actions

The notebook provides a strong foundation for several high-impact extensions:

*   **Build and Train ML Models**: The two views created (`Operational_Analyst_demand_forecasting_model` and `processing_time_prediction_model`) are feature-rich tables ready for machine learning. The next logical step is to use **BigQuery ML** to train a `ARIMA_PLUS` model for demand forecasting and a `BOOSTED_TREE_REGRESSOR` model for predicting processing times. This would shift the analysis from descriptive to predictive.
*   **Develop an Interactive Dashboard**: Connect this BigQuery project to **Looker Studio** or another BI tool. This would create a dynamic dashboard where stakeholders can filter by region, category, or date to explore the data and track KPIs in real-time.
*   **Actionable Business Recommendations**: Based on the validated insights, we can propose concrete actions:
    *   **Pricing Review**: Conduct a review of the pricing and discount strategy for the **Furniture** category, especially for loss-making sub-categories.
    *   **Regional Operational Audit**: Initiate an audit of the **Central** region's operations to identify the root causes of its low profitability.
    *   **Logistics Optimization**: Re-evaluate the carrier contracts and logistics for the **'Standard Class'** shipping mode to improve its speed and efficiency.
"""

file_name = "DIVE_Analysis.md"

with open(file_name, "w") as f:
    f.write(dive_content)

files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>